# Hard

- Tested On:
    - Ubuntu 18.04 x86 
        -pyvisa (backend: pyvisa-py)
    - Windows 10:
        -pyvisa (backend: NI-VISA)


## TODO:

Testing on more platforms:

- raspberrypi4.
- FreeBSD.

In [37]:
import time
import mhs5200

In [38]:
import platform

In [39]:
import os

In [40]:
if os.name == "posix":
    signal_gen_port = "/dev/ttyUSB0"
    oscilloscope_port = scope_port = 'USB0::6833::1416::DS1EU152500705\x00::0::INSTR'
if os.name == "nt":
    signal_gen_port = "COM4"
    oscilloscope_port = scope_port = 'USB0::0x1AB1::0x0588::DS1EU152500705::INSTR'

In [41]:
import mhs5200.enums

In [42]:
with mhs5200.MHS5200(signal_gen_port) as signal_generator:
    for channel in signal_generator.channels:
        channel.amplitude=1
        channel.duty_cycle=50
        channel.frequency=1
        channel.offset=0
        channel.phase=0
        channel.wave=mhs5200.enums.SQUARE

In [43]:
import pyvisa
rm = pyvisa.ResourceManager('@py')
print(rm.list_resources())

('ASRL/dev/ttyS4::INSTR', 'ASRL/dev/ttyUSB0::INSTR', 'USB0::6833::1416::DS1EU152500705\x00::0::INSTR')


In [44]:
import uuid

In [45]:
class Scope():
    def __init__(self, *args, oscilloscope=None, oscilloscope_port=None):
        if oscilloscope_port is not None:
            self.resource = rm.open_resource(oscilloscope_port)
        else:
            self.resource = oscilloscope
            
        for fcn in ["write", "query", "close", "query_ascii_values", "__repr__"]:
            setattr(self, fcn, getattr(self.resource, fcn))
        
    def __enter__(self):
        return self
    
    def __exit__(self, type, value, tb):
        t1=time.time()
        try:
            while True:
                if time.time()>t1+5:
                    break
                self.close()
                time.sleep(0.2)
        except InvalidSession:
            pass
        except:
            raise

In [46]:
import pandas as pd

In [47]:
import pandas as pd

In [48]:
class TestPair():
    def __init__(channel_output, channel_input):
        self.channel_output = channel_output
        self.channel_input  = channel_input

In [ ]:
df = pd.DataFrame()
with mhs5200.MHS5200(signal_gen_port) as signal_gen:
    with Scope(oscilloscope_port=oscilloscope_port) as scope:
        for _ in range(2):
            for frequency in np.logspace(np.log10(1000), np.log10(1000000), 2):
                for amplitude in [20]:
                    result_df = test_frequency_amplitude3(frequency, amplitude, signal_gen=signal_gen, scope=scope)
                    df = df.append(result_df)

In [74]:
def my_decorator(*args, **kwargs):
    # Args in passed to the decorator
    for idx, arg in enumerate(args):
        print(f"my_decorator {idx}: {arg}")
    for key,value in kwargs.items():
        print(f"my_decorator {key}: {value}") 
    # The inner function 
    def inner_function(*args2, **kwargs2):        
        for idx, arg in enumerate(args2):
            print(f"inner_function {idx}: {arg}")
        for key,value in kwargs2.items():
            print(f"inner_function {key}: {value}")
    return inner_function

In [75]:
class RigolChannel():
    def __init__(self, num, scope):
        self.num=num
        self.scope=scope
        
        
    def __repr__(self):
        return ""
        
    @property
    def source(self):
        return f"CHAN{self.num}"
    
    @my_decorator
    def fcn1(*args, **kwargs):        
        for idx, arg in enumerate(args):
            print(f"fcn1 {idx}: {arg}")
        for key,value in kwargs.items():
            print(f"fcn1 {key}: {value}")
        print("")     
    def __repr__(self):
        return "MyClass<!obj!>"

my_decorator 0: <function RigolChannel.fcn1 at 0x7fdfe9473790>


In [81]:
scope.close()

In [76]:
ch = RigolChannel(1, scope)

In [77]:
ch.fcn1()

inner_function 0: MyClass<!obj!>


In [78]:
self = ch

In [80]:
scope.write(f":MEASURE:SOURCE {self.source}")

InvalidSession: Invalid session handle. The resource might be closed.

In [72]:
ch.test1(1, 2, 4, foo="bar")

1
2
4
foo: bar


In [ ]:
for source in ["CHAN1", "CHAN2"]:
scope.write(f":MEASURE:SOURCE {source}")
time.sleep(1)
for param in ['VPP',
                'VMAX',
                'VMIN',
                'VAMPlitude',
                'VTOP',
                'VBASe',
                'VAVerage',
                'VRMS',
                'OVERshoot',
                'PREShoot',
                'FREQuency',
                'RISetime',
                'FALLtime',
                'PERiod',
                'PWIDth',
                'NWIDth',
                'PDUTycycle',
                'NDUTycycle',
                'PDELay',
                'NDELay',
                'TOTal',
                'SOURce',]:
    try:
        measured = scope.query_ascii_values(f":MEASURE:{param}?")[0]
        # Returns Junk
        if measured == 9.9e+37:
            measured = np.nan
    except:
        measured = scope.query(f":MEASURE:{param}?")[0]

In [33]:
def test_dataframe(frequency, amplitude, signal_gen, scope):
    for chan in signal_gen.channels:
        chan.frequency=frequency
        chan.amplitude=amplitude
        chan.phase=0
    period = 1/float(frequency)
    timescale="{:.20f}".format(float(period/5))
    # Configure scope
    
    scope.write(f":MEASURE:TOTAL ON")
    scope.write(f":TIMebase:SCALE {timescale}")
    for scope_channel in [1, 2]:
        scope.write(f":CHANNEL{scope_channel}:probe 1")
        scope.write(f":CHANNEL{scope_channel}:scale {amplitude/5}")
        scope.write(f":CHANNEL{scope_channel}:offset 0")
    # Configure signal generator
    for chan in signal_gen.channels:
        chan.frequency=frequency
        chan.amplitude=amplitude
        chan.offset = 0
        chan.phase=0

    df = dict()
    df["uuid"] = str(uuid.uuid4())
    df["frequency"] = frequency
    df["amplitude"] = amplitude
    for source in ["CHAN1", "CHAN2"]:
        scope.write(f":MEASURE:SOURCE {source}")
        time.sleep(1)
        for param in ['VPP',
                        'VMAX',
                        'VMIN',
                        'VAMPlitude',
                        'VTOP',
                        'VBASe',
                        'VAVerage',
                        'VRMS',
                        'OVERshoot',
                        'PREShoot',
                        'FREQuency',
                        'RISetime',
                        'FALLtime',
                        'PERiod',
                        'PWIDth',
                        'NWIDth',
                        'PDUTycycle',
                        'NDUTycycle',
                        'PDELay',
                        'NDELay',
                        'TOTal',
                        'SOURce',]:
            try:
                measured = scope.query_ascii_values(f":MEASURE:{param}?")[0]
                # Returns Junk
                if measured == 9.9e+37:
                    measured = np.nan
            except:
                measured = scope.query(f":MEASURE:{param}?")[0]

            df[f"{source}_{param}"] = measured
    return pd.DataFrame(df, index=[0])

In [ ]:
import numpy as np

In [ ]:
df = pd.DataFrame()
with mhs5200.MHS5200(signal_gen_port) as signal_gen:
    with Scope(oscilloscope_port=oscilloscope_port) as scope:
        for _ in range(2):
            for frequency in np.logspace(np.log10(1000), np.log10(1000000), 2):
                for amplitude in [20]:
                    result_df = test_frequency_amplitude3(frequency, amplitude, signal_gen=signal_gen, scope=scope)
                    df = df.append(result_df)

In [ ]:
df.groupby(["frequency"])["CHAN1_FREQuency"].agg(["mean", "median", "std", "var"])

In [25]:
import numpy as np

In [26]:
frequencies = np.logspace(np.log10(1000000), np.log10(100000000), 10)
amplitudes = [20]
n_tests=1

In [27]:
df = pd.DataFrame()
with mhs5200.MHS5200(signal_gen_port) as signal_gen:
    with Scope(oscilloscope_port=oscilloscope_port) as scope:
        for _ in range(n_tests):
            for frequency in frequencies:
                for amplitude in [20]:
                    result_df = test_frequency_amplitude3(frequency, amplitude, signal_gen=signal_gen, scope=scope)
                    df = df.append(result_df)
df.groupby(["frequency"])["CHAN1_FREQuency"].agg(["mean", "median", "std", "var"])

,mean,median,std,var
frequency,,,,
1.000000e+06,1.000000e+06,1.000000e+06,NaN,NaN
1.668101e+06,1.680000e+06,1.680000e+06,NaN,NaN
2.782559e+06,2.790000e+06,2.790000e+06,NaN,NaN
4.641589e+06,4.630000e+06,4.630000e+06,NaN,NaN
7.742637e+06,7.790000e+06,7.790000e+06,NaN,NaN
1.291550e+07,1.260000e+07,1.260000e+07,NaN,NaN
2.154435e+07,1.850000e+07,1.850000e+07,NaN,NaN
3.593814e+07,2.440000e+07,2.440000e+07,NaN,NaN
5.994843e+07,9.900000e+37,9.900000e+37,NaN,NaN


In [28]:
scope=Scope(oscilloscope_port=oscilloscope_port)

In [30]:
param="FREQUENCY"
scope.query_ascii_values(f":MEASURE:{param}?")[0]==9.9e+37

True

In [31]:
np.nan

nan

In [34]:
scope.close()

In [36]:
df = pd.DataFrame()
with mhs5200.MHS5200(signal_gen_port) as signal_gen:
    with Scope(oscilloscope_port=oscilloscope_port) as scope:
        test_frequency_amplitude3(100, 20, signal_gen, scope)